<a href="https://colab.research.google.com/github/dimitarpg13/rag_architectures_and_concepts/blob/main/src/examples/langchain//llm_caching/simple_sqlite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to cache LLM responses

LangChain provides an optional [caching](/docs/concepts/chat_models/#caching) layer for LLMs. This is useful for two reasons:

It can save you money by reducing the number of API calls you make to the LLM provider, if you're often requesting the same completion multiple times.
It can speed up your application by reducing the number of API calls you make to the LLM provider.


In [1]:
%pip install -qU langchain_openai langchain_community dotenv

import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass()


# Please manually enter OpenAI Key

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00


In [2]:
from langchain_core.globals import set_llm_cache
from langchain_openai import OpenAI

# To make the caching really obvious, let's use a slower and older model.
# Caching supports newer chat models as well.
llm = OpenAI(model="gpt-3.5-turbo-instruct", n=2, best_of=2)

In [3]:
%%time
from langchain_core.caches import InMemoryCache

set_llm_cache(InMemoryCache())

# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

CPU times: user 50 ms, sys: 12.8 ms, total: 62.7 ms
Wall time: 1.5 s


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired. "

In [4]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

CPU times: user 325 µs, sys: 70 µs, total: 395 µs
Wall time: 400 µs


"\n\nWhy couldn't the bicycle stand up by itself? Because it was two-tired. "

## SQLite Cache

In [5]:
!rm .langchain.db

rm: cannot remove '.langchain.db': No such file or directory


In [6]:
# We can do the same thing with a SQLite cache
from langchain_community.cache import SQLiteCache

set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [7]:
%%time
# The first time, it is not yet in cache, so it should take longer
llm.invoke("Tell me a joke")

CPU times: user 33.1 ms, sys: 2.05 ms, total: 35.2 ms
Wall time: 1.15 s


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"

In [8]:
%%time
# The second time it is, so it goes faster
llm.invoke("Tell me a joke")

CPU times: user 2.62 ms, sys: 110 µs, total: 2.73 ms
Wall time: 2.42 ms


"Why couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!"